In [1]:
import pickle
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))
from seq2seq import *
import pandas as pd
import numpy as np
import torch
import random
import itertools
from sklearn.preprocessing import StandardScaler

In [2]:
def set_seeds(seed):
    np.random.seed(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
# Convert a string that simulates a list to a real list
def convert_string_list(element):
    # Delete [] of the string
    element = element[0:len(element)]
    # Create a list that contains each code as e.g. 'A'
    ATC_list = list(element.split('; '))
    for index, code in enumerate(ATC_list):
        # Delete '' of the code
        ATC_list[index] = code[0:len(code)]
    return ATC_list

In [4]:
train_set = pd.read_csv('../../Data/train_set.csv')
test_set = pd.read_csv('../../Data/test_set.csv')
val_set = pd.read_csv('../../Data/val_set.csv')
set_seeds(78)

In [5]:
def multiplicate_rows(df):
    # Duplicate each compound the number of ATC codes associated to it, copying its SMILES in new rows
    new_rows = []
    
    for _, row in df.iterrows():
        atc_codes = row['ATC Codes']
        atc_codes_list = convert_string_list(atc_codes)
        
        if len(atc_codes_list) > 1:
            for code in atc_codes_list:
                if len(code) == 5:
                    new_row = row.copy()
                    new_row['ATC Codes'] = code
                    new_rows.append(new_row)
        else:
            if len(atc_codes_list[0]) == 5:
                new_rows.append(row)
    
    new_set = pd.DataFrame(new_rows)
    new_set = new_set.reset_index(drop=True)

    return new_set

#Extract molecular descriptors from the dataframe.
def extract_descriptors(df):
    descriptors = df.iloc[:, 2:-5].values
    
    # descriptors[pd.isinf(descriptors)] = pd.nan
    descriptors[pd.isna(descriptors)] = np.nanmedian(descriptors)

    return torch.tensor(descriptors, dtype=torch.float32)
    
# Create vocabularies
# Tokenize the data
def source(df):
    source = []
    for compound in df['Neutralized SMILES']:
        # A list containing each SMILES character separated
        source.append(list(compound))
    return source
def target(df):
    target = []
    for codes in df['ATC Codes']:  
        code = convert_string_list(codes) 
        # A list of lists, each one containing each ATC code character separated 
        for c in code:
            list_c = list(c)
            target.append(list_c)
    return target

In [6]:
new_train_set = multiplicate_rows(train_set)
new_val_set = multiplicate_rows(val_set)
new_test_set = multiplicate_rows(test_set)

new_test_set.to_csv("onecodeperdrug_test_set.csv", index = False)
new_val_set.to_csv("onecodeperdrug_val_set.csv", index = False)

train_descriptors = extract_descriptors(new_train_set)
test_descriptors = extract_descriptors(new_test_set)
test_descriptors2 = extract_descriptors(test_set)
val_descriptors = extract_descriptors(new_val_set)
val_descriptors2 = extract_descriptors(val_set)

scaler = StandardScaler()
train_descriptors = torch.tensor(scaler.fit_transform(train_descriptors.numpy()), dtype=torch.float32)
val_descriptors = torch.tensor(scaler.transform(val_descriptors.numpy()), dtype=torch.float32)
test_descriptors = torch.tensor(scaler.transform(test_descriptors.numpy()), dtype=torch.float32)
val_descriptors2 = torch.tensor(scaler.transform(val_descriptors2.numpy()), dtype=torch.float32)
test_descriptors2 = torch.tensor(scaler.transform(test_descriptors2.numpy()), dtype=torch.float32)

source_train = source(new_train_set)
source_test = source(new_test_set)
# Test set without duplicated compounds
source_test2 = source(test_set)
source_val = source(new_val_set)
# Val set without duplicated compounds
source_val2 = source(val_set)

target_train = target(new_train_set)
target_test = target(new_test_set)
target_val = target(new_val_set)

# An Index object represents a mapping from the vocabulary to integers (indices) to feed into the models
source_index = index.Index(source_train)
target_index = index.Index(target_train)

# Create tensors
X_train = source_index.text2tensor(source_train)
y_train = target_index.text2tensor(target_train)
X_val = source_index.text2tensor(source_val)
X_val2 = source_index.text2tensor(source_val2)
y_val = target_index.text2tensor(target_val)     
X_test = source_index.text2tensor(source_test)
X_test2 = source_index.text2tensor(source_test2)
y_test = target_index.text2tensor(target_test)

if torch.cuda.is_available():
    X_train = X_train.to("cuda")
    y_train = y_train.to("cuda")
    train_descriptors = train_descriptors.to("cuda") 
    test_descriptors = test_descriptors.to("cuda")
    test_descriptors2 = test_descriptors2.to("cuda")
    val_descriptors = val_descriptors.to("cuda")
    val_descriptors2 = val_descriptors2.to("cuda")
    X_val = X_val.to("cuda")
    X_val2 = X_val2.to("cuda")
    y_val = y_val.to("cuda")
    X_test= X_test.to("cuda")
    y_test = y_test.to("cuda")
    X_test2 = X_test2.to("cuda")

In [7]:
hyperparameters_grid = { 
    'enc_embedding_dim': [32, 64, 128],
    'dec_embedding_dim': [32, 64, 128],
    'enc_hidden_units': [32, 64, 128],
    'dec_hidden_units': [32, 64, 128],
    'enc_layers': [2, 3, 4],
    'dec_layers': [2, 3, 4],
    'dropout': [0.0, 0.1, 0.2],
    'weight_decays': [10**-4, 10**-5],
    'learning_rates': [10**-3, 10**-4]
}


In [8]:
# Randomly sample from dictionary
random_params = {k: random.sample(v, 1)[0] for k, v in hyperparameters_grid.items()}
print(random_params['enc_embedding_dim'])

64


In [9]:
def random_search(max_evals):
    tested_params = set()
    df_tests = pd.DataFrame(columns = ['#epochs', 'encoder_embedding_dim', 'decoder_embedding_dim', 'enc_layers', 'dec_layers', 'enc_hidden_units', 'dec_hidden_units', 'dropout', 'weight_decay', 'learning_rate', 'Precisionatn nivel1', 'Precisionatn nivel2', 'Precisionatn nivel3', 'Precisionatn nivel4', 'Precision nivel1', 'Precision nivel2', 'Precision nivel3', 'Precision nivel4', 'Recall nivel1', 'Recall nivel2', 'Recall nivel3', 'Recall nivel4', 'Drugs that have at least one match'], index = list(range(max_evals)))
    sys.stdout = open('log.txt', 'w')
    for i in range(max_evals):
        while True:
            random_params = {k: random.sample(v, 1)[0] for k, v in hyperparameters_grid.items()}
            params_tuple = tuple(random_params.values())
            if params_tuple not in tested_params:
                tested_params.add(params_tuple)
                break   
        model = multimodal_models.MultimodalBiLSTM(
                 source_index, 
                 target_index,
                 encoder_embedding_dimension = random_params['enc_embedding_dim'],
                 descriptors_dimension = train_descriptors.shape[1],
                 decoder_embedding_dimension = random_params['dec_embedding_dim'],
                 encoder_hidden_units = random_params['enc_hidden_units'],
                 encoder_layers = random_params['enc_layers'],
                 decoder_hidden_units = random_params['dec_hidden_units'],
                 decoder_layers = random_params['dec_layers'],
                 dropout = random_params['dropout'])   
        model.to("cuda")
        model.fit(
                X_train,
                train_descriptors,
                y_train,
                X_val, 
                val_descriptors,
                y_val, 
                batch_size = 32, 
                epochs = 500, 
                learning_rate = random_params['learning_rates'], 
                weight_decay = random_params['weight_decays'],
                progress_bar = 0, 
                save_path = None
        ) 
        model.load_state_dict(torch.load("best_multimodalmodel.pth", weights_only=True))
        ep = model.early_stopping.best_epoch
        loss, error_rate = model.evaluate(X_val, val_descriptors, y_val)    
        predictions, log_probabilities = search_algorithms.multimodal_beam_search(
            model, 
            X_val,
            val_descriptors,
            predictions = 6, # max length of the predicted sequence
            beam_width = 3,
            batch_size = 32, 
            progress_bar = 0
        )
        output_beam = [target_index.tensor2text(p) for p in predictions]
        predictions2, log_probabilities2 = search_algorithms.multimodal_beam_search(
            model, 
            X_val2,
            val_descriptors2,
            predictions = 6, # max length of the predicted sequence
            beam_width = 3,
            batch_size = 32, 
            progress_bar = 0
        )
        output_beam2 = [target_index.tensor2text(p) for p in predictions2]
        
        predictions_onecodeperdrug = []
        for preds in output_beam:
            interm = []
            for pred in preds:
                clean_pred = pred.replace('<START>', '').replace('<END>', '')
                if len(clean_pred) == 5:
                    interm.append(clean_pred)
            predictions_onecodeperdrug.append(interm)
                
        predictions = []
        for preds in output_beam2:
            interm = []
            for pred in preds:
                clean_pred = pred.replace('<START>', '').replace('<END>', '')
                if len(clean_pred) == 5:
                    interm.append(clean_pred)
            predictions.append(interm)
                
        precisionatn_1, precisionatn_2, precisionatn_3, precisionatn_4 = defined_metrics.precisionatn(predictions_onecodeperdrug, "onecodeperdrug_val_set.csv", 'ATC Codes')
        precision_1, precision_2, precision_3, precision_4 = defined_metrics.precision(predictions, "../../Data/val_set.csv", 'ATC Codes')
        recall_1, recall_2, recall_3, recall_4, comp = defined_metrics.recall(predictions, "../../Data/val_set.csv", 'ATC Codes')
        df_tests.iloc[i, :] = [f"{ep}", f"{random_params['enc_embedding_dim']}", f"{random_params['dec_embedding_dim']}", f"{random_params['enc_layers']}", f"{random_params['dec_layers']}", f"{random_params['enc_hidden_units']}", f"{random_params['dec_hidden_units']}", f"{random_params['dropout']}", f"{random_params['weight_decays']}", f"{random_params['learning_rates']}", f"{precisionatn_1}", f"{precisionatn_2}", f"{precisionatn_3}", f"{precisionatn_4}", f"{precision_1}", f"{precision_2}", f"{precision_3}", f"{precision_4}", f"{recall_1}", f"{recall_2}", f"{recall_3}", f"{recall_4}", f"{comp}"]
        df_tests.to_csv("multimodalbilstm_results.csv", index = False)
    sys.stdout = sys.__stdout__
    return df_tests

In [10]:
df_tests = random_search(200)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from matplotlib.lines import Line2D

df_res = pd.read_csv("multimodalbilstm_results.csv")
df_res = df_res[["encoder_embedding_dim", "decoder_embedding_dim", "enc_layers", "dec_layers", "enc_hidden_units", "dec_hidden_units", "dropout", 
                 "weight_decay", "learning_rate", "Precision nivel1", "Recall nivel1"]]

df_res['embedding dimension'] = df_res.apply(
    lambda row: f'enc_{int(row["encoder_embedding_dim"])} - dec_{int(row["decoder_embedding_dim"])}', axis=1)
emb_dim_map = {"enc_64 - dec_64": 'o', "enc_128 - dec_128": '^', "enc_64 - dec_128": 's', "enc_128 - dec_64": '*'}
df_res['embedding dimension'] = df_res['embedding dimension'].map(emb_dim_map)

df_res['encoder and decoder layers'] = df_res.apply(
    lambda row: f'enc_{int(row["enc_layers"])} - dec_{int(row["dec_layers"])}', axis=1)
df_res['weight_decay and learning_rate'] = df_res.apply(
    lambda row: f'WD_{row["weight_decay"]} - LR_{row["learning_rate"]}', axis=1)

size_map = {0.1: 100, 0.2: 300}
df_res['dropout value'] = df_res['dropout'].map(size_map)

df_res['encoder and decoder hidden units'] = df_res.apply(
    lambda row: f'enc_{int(row["enc_hidden_units"])} - dec_{int(row["dec_hidden_units"])}', axis=1)
df_res['edgewidth'] = df_res['encoder and decoder hidden units'].map({"enc_32 - dec_32": 0.5, "enc_32 - dec_64": 1.5,  "enc_64 - dec_32": 2.5, "enc_64 - dec_64": 3.5})

color_palette = sns.color_palette("Pastel1", n_colors=df_res['encoder and decoder layers'].nunique())

unique_wd_lr = df_res['weight_decay and learning_rate'].unique()
color_palette2 = sns.color_palette("Set1", n_colors=len(unique_wd_lr))
wd_lr_color_map = dict(zip(unique_wd_lr, color_palette2))

unique_enc_dec = df_res['encoder and decoder layers'].unique()
enc_dec_color_map = dict(zip(unique_enc_dec, color_palette))

plt.figure(figsize=(12, 12))

for i, row in df_res.iterrows():
    plt.scatter(
        x=row['Precision nivel1'],
        y=row['Recall nivel1'],
        s=row['dropout value'],  
        color=enc_dec_color_map[row['encoder and decoder layers']],  
        marker=row['embedding dimension'], 
        edgecolor=wd_lr_color_map[row['weight_decay and learning_rate']],  
        linewidth=row['edgewidth'],  
        alpha=0.7
    )
    
legend_elements = [
    Line2D([0], [0], color='none', label='Encoder and decoder embedding dimension'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='black', markersize=10, label='enc_64 - dec_64'),
    Line2D([0], [0], marker='^', color='w', markerfacecolor='black', markersize=10, label='enc_128 - dec_128'),
    Line2D([0], [0], marker='s', color='w', markerfacecolor='black', markersize=10, label='enc_64 - dec_128'),
    Line2D([0], [0], marker='*', color='w', markerfacecolor='black', markersize=10, label='enc_128 - dec_64'),
    Line2D([0], [0], color='black', linewidth=0.5, label='enc_32 - dec_32'),
    Line2D([0], [0], color='black', linewidth=1.5, label='enc_32 - dec_64'),
    Line2D([0], [0], color='black', linewidth=2.5, label='enc_64 - dec_32'),
    Line2D([0], [0], color='black', linewidth=3.5, label='enc_64 - dec_64'),
    Line2D([0], [0], color='none', label='Encoder - Decoder layers'),
    *[Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=label) for label, color in enc_dec_color_map.items()],
    Line2D([0], [0], color='none', label='Weight decay - Learning rate'),
    *[Line2D([0], [0], marker='o', color='w', markeredgecolor=color, markerfacecolor='none', markersize=10, label=label) for label, color in wd_lr_color_map.items()]
]

size_mapping = {
    100: "0.1",
    300: "0.2"
}

size_legend = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='black', markersize=size/18, label=f'{size_mapping.get(int(size))}') 
    for size in df_res['dropout value'].unique() 
]

plt.legend(title='Hiperparámetros', handles = legend_elements + [Line2D([0], [0], color='none', label='Dropout')] + size_legend, loc='upper left', bbox_to_anchor=(1.05, 1))

plt.title("Multimodal BiLSTM Hyperparameter optimization", fontsize=16)
plt.xlabel("Precision level 1", fontsize=14)
plt.ylabel("Recall level 1", fontsize=14)
plt.tight_layout()
plt.savefig('multimodalbilstm_hyperparams.png', bbox_inches='tight')
plt.show()

In [12]:
df_tests.sort_values(by = "Precision nivel1")

,#epochs,encoder_embedding_dim,decoder_embedding_dim,enc_layers,dec_layers,enc_hidden_units,dec_hidden_units,dropout,weight_decay,learning_rate,...,Precisionatn nivel4,Precision nivel1,Precision nivel2,Precision nivel3,Precision nivel4,Recall nivel1,Recall nivel2,Recall nivel3,Recall nivel4,Drugs that have at least one match
189,82,128,32,3,4,32,64,0.0,0.0001,0.0001,...,28.57142857142857,0.14125753660637397,0.330749354005168,0.46969696969696967,0.23015873015873017,0.30835486649440136,0.32945736434108525,0.4772727272727273,0.2857142857142857,"[129, 44, 21, 6]"
91,110,64,128,4,4,32,32,0.2,0.0001,0.0001,...,32.0,0.16192937123169693,0.3434004474272931,0.4294871794871795,0.2638888888888889,0.35831180017226527,0.33557046979865773,0.46153846153846156,0.3125,"[149, 52, 24, 8]"
39,120,128,64,3,4,64,32,0.1,1e-05,0.0001,...,37.03703703703704,0.1662360034453059,0.30666666666666664,0.5141843971631206,0.3148148148148148,0.35977605512489236,0.3,0.5638297872340425,0.37037037037037035,"[150, 47, 27, 10]"
190,91,32,32,3,4,64,32,0.0,0.0001,0.0001,...,13.636363636363635,0.1670973298880276,0.2358024691358025,0.5333333333333334,0.13636363636363635,0.3270456503014642,0.2851851851851852,0.5375,0.13636363636363635,"[135, 40, 22, 3]"
35,122,64,32,4,4,32,32,0.0,0.0001,0.0001,...,14.285714285714285,0.17054263565891492,0.16013071895424835,0.5,0.14285714285714285,0.3683893195521103,0.16666666666666666,0.5,0.14285714285714285,"[153, 28, 14, 2]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,21,64,64,2,2,32,128,0.1,1e-05,0.001,...,83.91608391608392,0.5676141257536605,0.7785547785547784,0.8187830687830687,0.6687853107344633,0.707249497559575,0.8659188034188035,0.9073034769463341,0.8498587570621469,"[286, 252, 236, 206]"
21,22,32,32,3,2,32,128,0.0,0.0001,0.001,...,77.431906614786,0.568906115417743,0.76,0.7921985815602839,0.5625,0.6785673270169394,0.8306565656565656,0.8872948328267478,0.7820987654320987,"[275, 235, 216, 175]"
146,29,32,128,2,3,64,128,0.1,0.0001,0.001,...,79.46768060836501,0.5736434108527131,0.749413145539906,0.7791842475386778,0.6131498470948009,0.7007464829170257,0.8186130672926447,0.8867691380349608,0.802446483180428,"[284, 237, 218, 180]"
116,49,64,64,3,4,64,128,0.2,0.0001,0.001,...,76.63934426229508,0.5770887166236002,0.7649253731343285,0.7186147186147186,0.6059870550161811,0.654364053976457,0.8419879767827528,0.8598639455782313,0.7892394822006473,"[268, 231, 206, 170]"


In [13]:
(df_tests.sort_values(by = "Precision nivel1")).to_csv("multimodalbilstm_sortedresults.csv", index = False)